In [2]:
import torch
from torch.utils import data
import random
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from collections import Counter
import pandas as pd
import numpy as np
import scipy
from tqdm import trange
from tqdm import tqdm
from datetime import datetime
import sys
import os
import seaborn as sns
from matplotlib import pyplot as plt
from joblib import Parallel, delayed, dump, load
from matplotlib import pyplot as plt
from sparse_vector.sparse_vector import SparseVector
from scipy.signal import convolve2d, convolve
import time
from torch import nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, f1_score
from IPython.display import clear_output

import torch
from transformers import BertModel, BertConfig, PreTrainedTokenizer, BasicTokenizer, BertForTokenClassification
import collections

from transformers import utils
from bertviz import model_view, head_view

from torch.utils.data import DataLoader
import sklearn
from sklearn.metrics import accuracy_score
from torch.nn import CrossEntropyLoss

import gc
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

In [3]:
from dna_tokenizer import DNATokenizer, seq2kmer

In [3]:
class Dataset(data.Dataset):
    def __init__(self, chroms, features, 
                 dna_source, features_source, 
                 labels_source, intervals, tokenizer):

        self.chroms = chroms
        self.features = features
        self.dna_source = dna_source
        self.features_source = features_source
        self.labels_source = labels_source
        self.intervals = intervals
        self.le = LabelBinarizer().fit(np.array([["A"], ["C"], ["T"], ["G"]]))
        self.configs = {
                        'ZHUNT_AS': {
                                'CG': 0, 'GC': 1, 'CA': 0, 'AC': 1, 
                                'TG': 0, 'GT': 1, 'TA': 1, 'AT': 1, 
                                'CC': 0, 'GG': 0, 'CT': 1, 'TC': 1, 
                                'GA': 1, 'AG': 1, 'AA': 1, 'TT': 1},
                       }
        seqs = (["A", "C", "T", "G"] + 
                ['AC', 'AT', 'AG', 'CT', 'CG', 'GT'] +
                ['AAC', 'ACC', 'AAT', 'ATT', 'AAG', 'AGG', 
                 'CCA', 'CAA', 'CCT', 'CTT', 'CCG', 'CGG', 
                 'TTA', 'TAA', 'TTC', 'TCC', 'TTG', 'TGG', 
                 'GGA', 'GAA', 'GGC', 'GCC', 'GGT', 'GTT'] +
                ['AAAC', 'AAAT', 'AAAG', 'CCCA', 'CCCT', 'CCCG',
                 'TTTA', 'TTTC', 'TTTG', 'GGGA', 'GGGC', 'GGGT'])
        self.tars = np.array([self.le.transform(list(i * 11)[:11]) for i in seqs])[:, ::-1, ::-1]
        # purine-pyrimidine
        self.tars = np.concatenate((self.tars, np.array([self.tars[4] + self.tars[9]])))
        self.tokenizer = tokenizer
        
        
    def __len__(self):
        return len(self.intervals)
    
    def __getitem__(self, index):
        interval = self.intervals[index]
        chrom = interval[0]
        begin = int(interval[1])
        end = int(interval[2])
        ll = list(self.dna_source[chrom][begin:end].upper())
        y = self.labels_source[interval[0]][interval[1]: interval[2]]        
        
        
#         DNA PART
        
        dna_OHE = self.le.transform(ll)[None]
        
        res = pd.DataFrame(convolve(dna_OHE, self.tars)[:, 5:-5, 3].T / 11)
        res = (res.rolling(5, min_periods=1).max().values == 1).astype(int)
        
        
#         ZHUNT PART
        zhunts = []
        for key in self.configs:
            vec = np.array(ll)
            vec = np.vectorize(lambda x:self.configs[key].get(x, 0))(
                                    np.char.add(vec[1:], vec[:-1]))
            zhunts.append(np.concatenate([vec, [0]]))
        
        
        # FEATURES PART
        feature_matr = []
        for feature in self.features:
            source = self.features_source[feature]
            feature_matr.append(source[chrom][begin:end])
        
        # UNION
        if len(feature_matr) > 0:
            X = np.hstack((
                           res,
                           np.array(zhunts).T, 
                           np.array(feature_matr).T/1000)).astype(np.float32)
#             X = (np.array(feature_matr).T/1000).astype(np.float32)
        else:
            X = dna_OHE.astype(np.float32)
        
        #K-mer part
        
        k_mers = seq2kmer(self.dna_source[chrom][begin:end+5].upper(),6)
        encoded_k_mers = self.tokenizer.encode_plus(k_mers, add_special_tokens=False, max_length=512)["input_ids"]

        return torch.Tensor(X), torch.Tensor(y).long(), ll, torch.LongTensor(encoded_k_mers), (chrom, begin, end)

In [ ]:
from collections import defaultdict
kmer2pred = defaultdict(int)
kmer2att = defaultdict(float)

device = 3
for FOLD in range(5):
    gc.collect()
    attentions, preds, targets, seqs, bps = [],[],[], [], []
    train_dataset, test_dataset = load(f'/gim/lv01/dumerenkov/zdna_data/datasets/ds_w_seq_mm_fold{FOLD}_kouzine.pkl')
    model = BertForTokenClassification.from_pretrained(f'dnabert_mm_fold_{FOLD}_kouzine', output_attentions=True)
    model.to(device)
    for example in tqdm(test_dataset):
        features, target, seq, input_ids, interval = example
        if target.numpy().sum()>0:
            with torch.no_grad():
                input_ids = input_ids.to(device)
                outputs = model(input_ids.unsqueeze(0))
                
                raw_preds = outputs[-2].detach().to('cpu')
                attention = outputs[-1][-1].detach().to('cpu')
                
                pred = torch.softmax(raw_preds, axis = -1)[0,:,1]

                
            attentions.append(attention)
            preds.append(pred)
            targets.append(target)
            seqs.append(seq)
            
    for attention, pred, target, seq in tqdm(zip(attentions, preds, targets, seqs)):
        kmer = seq2kmer(''.join(seq), 6).split(' ')
        #print(kmer)
        att = attention[0,:,:,:]
    
        for idx in range(512-5):
            if target[idx]>0:
                kmer2pred[kmer[idx]]+=1
            
                for head in range(12):
                    c_att = att[head,idx,:].numpy()
                    for att_idx in range(512-5):
                        kmer2att[kmer[att_idx]]+=c_att[att_idx]
    
    

In [7]:
sorted_pred = [t[0] for t in sorted(kmer2pred.items(), key=lambda item: -item[1])]
kmer2att2 = {k:int(kmer2att[k]) for k in kmer2att}
sorted_att  = [t[0] for t in sorted(kmer2att2.items(), key=lambda item: -item[1])]

In [8]:
for idx, kmer in enumerate(sorted_att):
    print(idx+1, kmer, sorted_pred.index(kmer)+1)
    if idx>100:
        break

1 CACACA 2
2 TGTGTG 1
3 GTGTGT 3
4 ACACAC 4
5 CGCGCG 6
6 GCGCGC 5
7 GTGTGC 7
8 ACACAT 9
9 ATGTGT 11
10 GCACAC 8
11 ATACAC 65
12 GTGTAT 43
13 TACACA 61
14 GGCGCG 12
15 TGCGTG 14
16 GTATGT 63
17 CACACG 27
18 ACATAC 55
19 CATACA 52
20 CACATA 38
21 CACGCA 19
22 CGTGTG 16
23 CGCGCC 10
24 GTGCGT 31
25 TGTGCG 24
26 CGCACA 17
27 TGTGTA 23
28 GTGCAT 54
29 TATGTG 67
30 GTGCGC 15
31 TGTATG 56
32 CCGCGC 20
33 TGTGCA 26
34 CACATG 35
35 CGCGCA 18
36 TGCACA 29
37 TGCATG 36
38 TGCGCG 25
39 GGGCGC 32
40 CCCGCG 41
41 ACGCAC 33
42 CATGTG 30
43 ATGCAC 62
44 GCGCAC 13
45 GCACGC 22
46 CGCGGG 44
47 CGCACG 42
48 GCGCGG 21
49 CACGCG 49
50 CATGCA 39
51 ACACGC 40
52 GCGTGC 28
53 ACACAG 68
54 CGTGCG 48
55 CGGGCG 59
56 CGCGTG 53
57 GCGTGT 37
58 GCGCCC 34
59 GCCCGC 51
60 ACATGC 46
61 CTCACA 105
62 CTGTGT 80
63 GCATGT 45
64 TGCGCA 47
65 GTCTGT 104
66 CGCCCG 57
67 ACTCAC 106
68 GTGCAC 66
69 GCGGGC 50
70 ACGCGC 58
71 GGTGTG 99
72 CACAGA 75
73 GTGAGT 136
74 GACACA 121
75 CACACT 69
76 GTGTCT 86
77 CACACC 71
78 GTGTGA 76

In [4]:
width = 512
k_mer_pad = 5

def final_prediction(chrom):
    
    intervals = []
    ends = []
    
    prediction = [np.zeros(ZDNA[chrom].shape, dtype=np.float32) for i in range(5)]
    
    
    for st in range(0, ZDNA[chrom].shape[0] - width, width):
        interval = [st, min(st + width + k_mer_pad, ZDNA[chrom].shape[0])]
        intervals.append([chrom, interval[0], interval[1]])
        
    ends.append([chrom, ZDNA[chrom].shape[0] // width * width, ZDNA[chrom].shape[0]])


    full_dataset = Dataset(chroms, [], 
                           DNA, [],  ZDNA, intervals, tokenizer)

    ends_dataset = Dataset(chroms, [], 
                           DNA, [], ZDNA, ends, tokenizer)

    params = {'batch_size':32, 'num_workers':5, 'shuffle':False}
    load_predict = data.DataLoader(full_dataset, **params)

    params = {'batch_size':1, 'num_workers':1, 'shuffle':False}
    load_ends = data.DataLoader(ends_dataset, **params)
    
    with torch.no_grad():
        
        for features, labels, sequences, input_ids, intervals in load_predict:
            input_ids = input_ids.to(device)
            outputs = [torch.softmax(model(input_ids = input_ids)['logits'],axis = -1).cpu().numpy()[:,:,1] for model in models]
            
            for ind, interval in enumerate(zip(intervals[0], 
                                         intervals[1].cpu().numpy(),
                                         intervals[2].cpu().numpy())):
                #print(ind, interval)
                #print(len(prediction), len(outputs), len(models))
                for i in range(len(models)):
                    prediction[i][interval[1]:interval[2]-k_mer_pad] = outputs[i][ind, :]
                    
    for model_i in range(len(models)):
        dump(prediction[model_i], f'new_mod_mm_{model_i}_{chrom}_kouzine', 3)

In [9]:
device = 1
torch.cuda.empty_cache()
models = []

config = BertConfig.from_pretrained('https://raw.githubusercontent.com/jerryji1993/DNABERT/master/src/transformers/dnabert-config/bert-config-6/config.json')

for MODEL_NUMBER in range(5):
    dir_to_pretrained_model = f'dnabert_mm_fold_{MODEL_NUMBER}_kouzine'
    model = BertForTokenClassification.from_pretrained(dir_to_pretrained_model, config=config).to(device)
    model.to(device)
    model.eval()
    models.append(model)

In [3]:
chroms = [f'chr{i}' for i in list(range(1, 20)) + ['X', 'Y']]
DNA = {chrom:load(f'../data/mm9_dna/sparse/{chrom}.pkl') for chrom in tqdm(chroms)}
ZDNA = load(f'ZDNA_mm9_kouzine.pkl')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:31<00:00,  1.50s/it]


In [11]:
from dna_tokenizer import DNATokenizer, seq2kmer
tokenizer = DNATokenizer.from_pretrained('6-new-12w-0/', add_special_tokens=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DNATokenizer'.


In [12]:
for chrom in chroms[:]:
    print(f"BEGIN CHROM {chrom}")
    final_prediction(chrom)

BEGIN CHROM chr1


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

BEGIN CHROM chr2


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

BEGIN CHROM chr3


...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

BEGIN CHROM chrX


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

BEGIN CHROM chrY


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-s

In [13]:
equalized, divisions = load('mm_divisions_kouzine.pkl')

In [19]:
com_len = sum([len(DNA[chrom]) for chrom in chroms])
sums = []

for chrom in tqdm(chroms):
    loc_sum = []
    for model_num in range(5):
        vec = load(f"preds/new_mod_mm_{model_num}_{chrom}_kouzine")
        loc_sum.append(vec.sum())
    sums.append(loc_sum)

multipliers = np.array(sums).sum(axis=0) / com_len

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [06:08<00:00, 17.52s/it]


In [20]:
for chrom in tqdm(chroms):
    vecs = np.array([load(f"preds/new_mod_mm_{model_num}_{chrom}_kouzine") 
                     for model_num in range(5)])
    res_vec = (vecs / multipliers[:, None]) * multipliers.mean()
    mean_vec = res_vec.mean(axis=0)
    
    test_ints = []
    for MODEL_NUMBER in range(5):
        train_inds, test_inds = divisions[MODEL_NUMBER]
        train_intervals, test_intervals = [equalized[i] for i in train_inds], [equalized[i] for i in test_inds]
        test_ints.extend([(MODEL_NUMBER, inter) for inter in test_intervals if inter[0] == chrom])
    
    for model_num, inters in test_ints:
        mean_vec[inters[1]: inters[2]] = res_vec[model_num, inters[1]: inters[2]]
    dump(mean_vec, f"preds/new_mod_mm_res_{chrom}_kouzine", 3)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [21:53<00:00, 62.53s/it]


In [22]:
blacklist = {c:[] for c in chroms}
with open("mm9-blacklist.bed")as f:
    for line in f:
        chrom, start, end =  line.strip().split()
        blacklist[chrom].append((int(start), int(end)))
        
for chrom in tqdm(chroms):
    vec = load(f"preds/new_mod_mm_res_{chrom}_kouzine")
    for s_idx, e_idx in blacklist[chrom]:
        vec[s_idx:e_idx] = 0
    dump(vec, f"preds/new_mod_mm_res_{chrom}_kouzine", 3)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [15:31<00:00, 44.35s/it]


In [3]:
chroms = [f'chr{i}' for i in list(range(1, 20)) + ['X', 'Y']]
ZDNA = load(f'ZDNA_mm9_kouzine.pkl')
blacklist = {c:[] for c in chroms}
with open("mm9-blacklist.bed")as f:
    for line in f:
        chrom, start, end =  line.strip().split()
        blacklist[chrom].append((int(start), int(end)))

all_pred = []
all_true = []
for chrom in tqdm(chroms):
    true_clean = ZDNA[chrom][:].astype(int)
    for s_idx, e_idx in blacklist[chrom]:
        true_clean[s_idx:e_idx] = 0
    all_pred.append(load(f"preds/new_mod_mm_res_{chrom}_kouzine"))
    all_true.append(true_clean)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [02:09<00:00,  6.15s/it]


In [3]:
roc_auc_score(np.concatenate(all_true), np.concatenate(all_pred))

0.9958508489086163

In [4]:
print(sklearn.metrics.classification_report(np.concatenate(all_true), np.concatenate(all_pred)>0.5, digits=4))

              precision    recall  f1-score   support

           0     0.9999    0.9966    0.9983 2654285742
           1     0.0490    0.7587    0.0921    609476

    accuracy                         0.9966 2654895218
   macro avg     0.5245    0.8777    0.5452 2654895218
weighted avg     0.9997    0.9966    0.9981 2654895218

